### 🧮 Agrupamientos y Agregaciones

Has aprendido a seleccionar y filtrar datos. Ahora es el momento de empezar a resumirlos para extraer insights valiosos. La agrupación de datos es una de las funcionalidades más potentes de Pandas y se basa en un paradigma llamado **Split-Apply-Combine** (Dividir-Aplicar-Combinar).

1.  **Split (Dividir):** El `DataFrame` se divide en grupos más pequeños basados en los valores de una o más columnas.
2.  **Apply (Aplicar):** Se aplica una función a cada uno de esos grupos de forma independiente (por ejemplo, calcular la media, sumar, contar, etc.).
3.  **Combine (Combinar):** Los resultados de aplicar la función a cada grupo se combinan en una nueva estructura de datos (normalmente un `DataFrame` o una `Serie`).

* **4.1. El Poder de `.groupby()`**

El método `.groupby()` es el que inicia este proceso. Por sí solo, no calcula nada, sino que crea un objeto especial `DataFrameGroupBy` que está listo para que le apliques una función de agregación.

In [1]:
import pandas as pd

# Cargamos de nuevo el dataset de pingüinos
url = 'https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv'
df = pd.read_csv(url)

# Agrupemos los datos por la columna 'species'
grupos_por_especie = df.groupby('species')

# ¿Qué es este objeto?
print(grupos_por_especie)

Como ves, no es un DataFrame. Es un objeto que contiene la información sobre los grupos. Ahora, apliquemos una función.

In [2]:
# Apply: Calculemos la media de todas las columnas numéricas para cada grupo
# Agregamos numeric_only=True para evitar un error con las columnas no numéricas
media_por_especie = grupos_por_especie.mean(numeric_only=True)

print("Media de las variables numéricas por especie:")
display(media_por_especie)

Media de las variables numéricas por especie:


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
species,,,,,
Adelie,38.791391,18.346358,189.953642,3700.662252,2008.013158
Chinstrap,48.833824,18.420588,195.823529,3733.088235,2007.970588
Gentoo,47.504878,14.982114,217.186992,5076.016260,2008.080645


**Nota sobre el error `TypeError`:** Si ejecutas `grupos_por_especie.mean()` sin más, en versiones recientes de Pandas podrías obtener un `TypeError`. Esto se debe a que el DataFrame contiene columnas no numéricas (como `island` y `sex`) y Pandas ya no intenta adivinar si debe ignorarlas. La solución correcta es ser explícito y usar `numeric_only=True` para indicarle que solo calcule la media de las columnas que son numéricas.

¡Magia! Pandas dividió el DataFrame en 3 grupos (Adelie, Chinstrap, Gentoo), calculó la media de las columnas numéricas para cada uno y combinó los resultados en un nuevo DataFrame.

Puedes ser más específico y calcular la agregación solo para una columna.

In [3]:
# Media de la masa corporal ('body_mass_g') por especie
media_masa_por_especie = df.groupby('species')['body_mass_g'].mean()

print("Media de la masa corporal por especie:")
display(media_masa_por_especie)

Media de la masa corporal por especie:


species
Adelie       3700.662252
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

* **4.2. Agregaciones Múltiples con `.agg()`**

¿Y si quieres calcular varias métricas a la vez? Para eso está el método `.agg()`. Puedes pasarle una lista de funciones a una sola columna:

In [4]:
# Para la columna 'flipper_length_mm', calculemos la media, la mediana y la desviación estándar
agregaciones_aleta = df.groupby('species')['flipper_length_mm'].agg(['mean', 'median', 'std'])

display(agregaciones_aleta)

,mean,median,std
species,,,
Adelie,189.953642,190.0,6.539457
Chinstrap,195.823529,196.0,7.131894
Gentoo,217.186992,216.0,6.484976


**Sintaxis Avanzada con Diccionarios**

Para un control aún mayor, puedes usar un diccionario para aplicar **diferentes funciones a diferentes columnas**. ¡Incluso puedes aplicar múltiples funciones a la misma columna!

In [5]:
# Usando un diccionario para especificar las agregaciones
# Para 'bill_length_mm' queremos la mínima y máxima
# Para 'body_mass_g' queremos la media
# Para 'sex' queremos contar cuántos hay

agregaciones_dict = {
    'bill_length_mm': ['min', 'max'],
    'body_mass_g': 'mean',
    'sex': 'count'
}

resumen_con_dict = df.groupby('species').agg(agregaciones_dict)

display(resumen_con_dict)

bill_length_mm        body_mass_g   sex
                     min   max         mean count
species                                          
Adelie              32.1  46.0  3700.662252   146
Chinstrap           40.9  58.0  3733.088235    68
Gentoo              40.9  59.6  5076.016260   119

*Nota*: Fíjate que el resultado tiene columnas con múltiples niveles (un `MultiIndex`). Esto es potente, pero a veces queremos un resultado "plano". Para eso, la siguiente sintaxis es la más recomendada.

**La Mejor Práctica: Agregaciones Nombradas**

La sintaxis más moderna y legible te permite aplicar diferentes funciones a diferentes columnas y, lo más importante, **nombrar las columnas resultantes directamente**. Esto evita el `MultiIndex` y hace el código más fácil de leer.

In [6]:
# Agrupemos por especie e isla, y calculemos métricas específicas
resumen_detallado = df.groupby(['species', 'island']).agg(
    conteo_pingüinos=('sex', 'count'),
    media_masa_corporal=('body_mass_g', 'mean'),
    max_largo_pico=('bill_length_mm', 'max'),
    min_largo_pico=('bill_length_mm', 'min')
)

display(resumen_detallado)

conteo_pingüinos  media_masa_corporal  max_largo_pico  \
species   island                                                             
Adelie    Biscoe                   44          3709.659091            45.6   
          Dream                    55          3688.392857            44.1   
          Torgersen                47          3706.372549            46.0   
Chinstrap Dream                    68          3733.088235            58.0   
Gentoo    Biscoe                  119          5076.016260            59.6   

                     min_largo_pico  
species   island                     
Adelie    Biscoe               34.5  
          Dream                32.1  
          Torgersen            33.5  
Chinstrap Dream                40.9  
Gentoo    Biscoe               40.9

* **4.3. Ordenando los Resultados con `.sort_values()`**

A menudo, querrás ordenar los resultados de tus agregaciones para ver los valores más altos o más bajos.

In [7]:
# Usemos el resumen anterior y ordenémoslo por la media de la masa corporal
# de forma descendente (los más pesados primero)

resumen_ordenado = resumen_detallado.sort_values(by='media_masa_corporal', ascending=False)

display(resumen_ordenado)

conteo_pingüinos  media_masa_corporal  max_largo_pico  \
species   island                                                             
Gentoo    Biscoe                  119          5076.016260            59.6   
Chinstrap Dream                    68          3733.088235            58.0   
Adelie    Biscoe                   44          3709.659091            45.6   
          Torgersen                47          3706.372549            46.0   
          Dream                    55          3688.392857            44.1   

                     min_largo_pico  
species   island                     
Gentoo    Biscoe               40.9  
Chinstrap Dream                40.9  
Adelie    Biscoe               34.5  
          Torgersen            33.5  
          Dream                32.1

* **4.4. ¿A Dónde se Fue mi Columna? `.reset_index()`**

Fíjate en los DataFrames que hemos creado con `groupby`. Las columnas por las que agrupamos (`species`, `island`) no son columnas normales, ¡son el **índice** del DataFrame!

Esto es útil, pero a veces quieres que vuelvan a ser columnas para poder filtrarlas o usarlas en gráficos. Para eso sirve `.reset_index()`.

In [8]:
# El índice de nuestro resumen son 'species' e 'island'
print("Índice ANTES de reset_index:")
print(resumen_ordenado.index)

# Convertimos el índice en columnas
resumen_final = resumen_ordenado.reset_index()

print("\nDataFrame DESPUÉS de reset_index:")
display(resumen_final.head())

Índice ANTES de reset_index:
MultiIndex([(   'Gentoo',    'Biscoe'),
            ('Chinstrap',     'Dream'),
            (   'Adelie',    'Biscoe'),
            (   'Adelie', 'Torgersen'),
            (   'Adelie',     'Dream')],
           names=['species', 'island'])

DataFrame DESPUÉS de reset_index:


,species,island,conteo_pingüinos,media_masa_corporal,max_largo_pico,min_largo_pico
0,Gentoo,Biscoe,119,5076.016260,59.6,40.9
1,Chinstrap,Dream,68,3733.088235,58.0,40.9
2,Adelie,Biscoe,44,3709.659091,45.6,34.5
3,Adelie,Torgersen,47,3706.372549,46.0,33.5
4,Adelie,Dream,55,3688.392857,44.1,32.1


Ahora `species` e `island` son columnas normales y el DataFrame tiene un nuevo índice numérico simple.

* **🧠 Ejercicios Propuestos**

Volvemos al Titanic. `url_titanic = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'`

1. **Carga los datos.**

2. **Agregación Simple:** Calcula la edad (`Age`) promedio de los pasajeros agrupando por clase (`Pclass`). ¿En qué clase viajaban los pasajeros de mayor edad en promedio?

3. **Agregación por Múltiples Columnas:** Calcula la tarifa (`Fare`) promedio pagada, agrupando por clase (`Pclass`) y sexo (`Sex`).

4. **Agregaciones Múltiples con `.agg()`:**

   * Agrupa los datos por la columna `Survived` (0 = No, 1 = Sí).

   * Para cada grupo, calcula la edad media (`mean`), la tarifa máxima (`max`) y la tarifa mínima (`min`).

   * Usa la sintaxis de **agregaciones nombradas** para las nuevas columnas.

5. **Desafío Completo:** Encuentra la tasa de supervivencia por clase. Para ello, agrupa por `Pclass` y calcula la media de la columna `Survived` (como `Survived` es 0 o 1, la media es la proporción o tasa de supervivencia). Ordena el resultado para ver qué clase tuvo la mayor tasa de supervivencia.